In [575]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import catboost
from sklearn.model_selection import cross_val_score
from catboost import cv, Pool

In [576]:
#Functions

def fill_na_rank(name):
    if(len(np.concatenate([np.array(data.loc[(data['First_Rank'].notnull()) &
                                          (data['First_player']==name) &
                                          ~(data['First_Rank']=='NR'),'First_Rank'].ravel()),
                        np.array(data.loc[(data['Second_Rank'].notnull()) & 
                                          (data['Second_player']==name) &
                                          ~(data['Second_Rank']=='NR'),"Second_Rank"].ravel())]))==0):
        return np.nan
    return int(np.concatenate([np.array(data.loc[(data['First_Rank'].notnull()) &
                                          (data['First_player']==name) &
                                          ~(data['First_Rank']=='NR'),'First_Rank'].ravel()),
                        np.array(data.loc[(data['Second_Rank'].notnull()) & 
                                          (data['Second_player']==name) &
                                          ~(data['Second_Rank']=='NR'),"Second_Rank"].ravel())])\
        .mean())



In [577]:
#Reading data
df_2000 = pd.read_excel("2000.xls")
df_2001 = pd.read_excel("2001.xls")
df_2002 = pd.read_excel("2002.xls")
df_2003 = pd.read_excel("2003.xls")
df_2004 = pd.read_excel("2004.xls")
df_2005 = pd.read_excel("2005.xls")
df_2006 = pd.read_excel("2006.xls")
df_2007 = pd.read_excel("2007.xls")
df_2008 = pd.read_excel("2008.xls")
df_2009 = pd.read_excel("2009.xls")
df_2010 = pd.read_excel("2010.xls")
df_2011 = pd.read_excel("2011.xls")
df_2012 = pd.read_excel("2012.xls")
df_2013 = pd.read_excel("2013.xlsx")
df_2014 = pd.read_excel("2014.xlsx")
df_2015 = pd.read_excel("2015.xlsx")
df_2016 = pd.read_excel("2016.xlsx")
df_2017 = pd.read_excel("2017.xlsx")
df_2018 = pd.read_excel("2018.xlsx")
df_2019 = pd.read_excel("2019.xlsx")
df_2020 = pd.read_excel("2020.xlsx")

In [578]:
frames = [df_2000, df_2001, df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008,
         df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, 
          df_2018, df_2019, df_2020]

In [579]:
useful_columns = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'Wsets', 'Lsets', 'Comment']

In [580]:
#Aggregate into one
data = pd.concat(frames)

/home/yuri/Ozon/OzonML/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [581]:
data = data[useful_columns]

In [582]:
data.isnull().values.ravel().sum()

615

In [583]:
data = data.drop(columns="Tournament")

In [584]:
data = data.drop(columns="Location")

In [585]:
data['Series'].unique()

array(['International', 'Grand Slam', 'International Gold', 'Masters',
       'Masters Cup', 'ATP250', 'ATP500', 'Masters 1000'], dtype=object)

In [586]:
data["Best of"].unique()

array([3, 5])

In [587]:
data["Surface"].unique()

array(['Hard', 'Clay', 'Grass', 'Carpet'], dtype=object)

In [588]:
data["Round"].unique()

array(['1st Round', '2nd Round', 'Quarterfinals', 'Semifinals',
       'The Final', '3rd Round', '4th Round', 'Round Robin'], dtype=object)

In [589]:
data["Comment"].unique()

array(['Completed', 'Retired', 'Walkover', 'Disqualified', 'Sched',
       'Awarded'], dtype=object)

In [590]:
data["ATP"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69])

In [591]:
data = data.rename(columns={'Winner':'First_player', 'Loser':'Second_player', 'WRank':'First_Rank',
                            'LRank':'Second_Rank', 'Wsets':'First_Sets', 'Lsets':'Second_Sets'})

In [592]:
data.count()

ATP              55460
Date             55460
Series           55460
Court            55460
Surface          55460
Round            55460
Best of          55460
First_player     55460
Second_player    55460
First_Rank       55440
Second_Rank      55364
First_Sets       55211
Second_Sets      55210
Comment          55460
dtype: int64

In [593]:
data["Second_Rank"][data['Second_Rank']=="NR"]

63      NR
377     NR
560     NR
611     NR
613     NR
618     NR
774     NR
1039    NR
1694    NR
1849    NR
577     NR
588     NR
1494    NR
2472    NR
2524    NR
2528    NR
2531    NR
2535    NR
2565    NR
72      NR
381     NR
851     NR
1370    NR
1488    NR
1492    NR
2185    NR
816     NR
Name: Second_Rank, dtype: object

In [594]:
data["First_Rank"][(data['First_Rank'].isna())]

2002    NaN
2020    NaN
5       NaN
17      NaN
325     NaN
335     NaN
1473    NaN
319     NaN
70      NaN
275     NaN
2224    NaN
836     NaN
856     NaN
1391    NaN
1685    NaN
58      NaN
75      NaN
79      NaN
1631    NaN
388     NaN
Name: First_Rank, dtype: object

In [595]:
data['Second_Rank'][(data['Second_Rank'].isna())]

2040    NaN
27      NaN
324     NaN
1183    NaN
2139    NaN
       ... 
55      NaN
131     NaN
319     NaN
326     NaN
396     NaN
Name: Second_Rank, Length: 96, dtype: object

In [596]:
data.loc[(data['First_Rank'].isna()) | (data['First_Rank']=="NR"),'First_Rank'] = \
data["First_player"][(data['First_Rank'].isna()) | (data['First_Rank']=="NR")]\
.apply(lambda x: fill_na_rank(x))

In [597]:
data.loc[(data['Second_Rank'].isna()) | (data['Second_Rank']=="NR"),'Second_Rank'] = \
data["Second_player"][data['Second_Rank'].isna() | (data['Second_Rank']=="NR")]\
.apply(lambda x: fill_na_rank(x))

In [598]:
data['Second_Rank'] = data['Second_Rank'].fillna(int(data['Second_Rank'].mean()))

In [599]:
data['First_Rank'] = data['First_Rank'].fillna(int(data['First_Rank'].mean()))

In [600]:
data['Second_Sets'] = data['Second_Sets'].fillna(0)

In [601]:
data['First_Sets'] = data['First_Sets'].fillna(0)

In [602]:
data['First_Sets'] = data['First_Sets'].apply(lambda x: int(x))

In [603]:
data = data.reset_index(drop=True)

In [604]:
data['First_won'] = 1

In [605]:
#Shuffle between first and second

idx = (data.index < 28000)

data.loc[idx,['First_player','Second_player']] = data.loc[idx,['Second_player','First_player']].values
data.loc[idx,['First_Rank','Second_Rank']] = data.loc[idx,['Second_Rank','First_Rank']].values
data.loc[idx,['First_Sets','Second_Sets']] = data.loc[idx,['Second_Sets','First_Sets']].values
data.loc[idx, ['First_won']] = 0

In [606]:
data = data.sample(frac=1, random_state=420).reset_index(drop=True)

In [607]:
data

,ATP,Date,Series,Court,Surface,Round,Best of,First_player,Second_player,First_Rank,Second_Rank,First_Sets,Second_Sets,Comment,First_won
0,32,2014-05-31,Grand Slam,Outdoor,Clay,3rd Round,5,Monfils G.,Fognini F.,28.0,15.0,3,2,Completed,1
1,6,2009-01-23,Grand Slam,Outdoor,Hard,3rd Round,5,Delic A.,Djokovic N.,127.0,3.0,1,3,Completed,0
2,66,2018-10-30,Masters 1000,Indoor,Hard,2nd Round,3,Khachanov K.,Ebden M.,18.0,39.0,1,0,Retired,1
3,35,2009-06-18,ATP250,Outdoor,Grass,Quarterfinals,3,Chardy J.,Schuettler R.,43.0,30.0,1,2,Completed,0
4,19,2019-03-09,Masters 1000,Outdoor,Hard,2nd Round,3,Raonic M.,Querrey S.,14.0,51.0,2,0,Completed,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55455,66,2016-11-17,Masters Cup,Indoor,Hard,Round Robin,3,Raonic M.,Thiem D.,4.0,9.0,2,0,Completed,1
55456,53,2018-08-30,Grand Slam,Outdoor,Hard,2nd Round,5,Sousa J.,Carreno Busta P.,68.0,12.0,2,2,Retired,1
55457,19,2007-03-12,Masters,Outdoor,Hard,3rd Round,3,Korolev E.,Djokovic N.,71.0,13.0,0,2,Completed,0
55458,33,2019-06-11,ATP250,Outdoor,Grass,1st Round,3,Goffin D.,Davidovich Fokina A.,33.0,129.0,2,0,Completed,1


In [608]:
data[(data['First_player']=='Amritraj P.')|(data['Second_player']=='Amritraj P.')]

,ATP,Date,Series,Court,Surface,Round,Best of,First_player,Second_player,First_Rank,Second_Rank,First_Sets,Second_Sets,Comment,First_won
781,39,2007-07-13,International,Outdoor,Grass,Quarterfinals,3,Amritraj P.,Mahut N.,467.0,83.0,0,2,Completed,0
1758,39,2007-07-12,International,Outdoor,Grass,2nd Round,3,Jenkins S.,Amritraj P.,367.0,467.0,0,2,Completed,0
2398,2,2006-01-04,International,Outdoor,Hard,2nd Round,3,Amritraj P.,Stepanek R.,267.0,20.0,1,2,Completed,0
3606,40,2008-07-12,International,Outdoor,Grass,Semifinals,3,Dancevic F.,Amritraj P.,99.0,305.0,1,2,Completed,0
4681,39,2003-07-07,International,Outdoor,Grass,1st Round,3,Amritraj P.,Ginepri R.,414.0,54.0,0,2,Completed,0
5289,2,2006-01-02,International,Outdoor,Hard,1st Round,3,Mello R.,Amritraj P.,111.0,267.0,1,2,Completed,0
6715,42,2003-07-22,International,Outdoor,Hard,1st Round,3,Roitman S.,Amritraj P.,140.0,396.0,1,2,Completed,0
10777,45,2003-07-30,International,Outdoor,Hard,2nd Round,3,Amritraj P.,Carlsen K.,351.0,76.0,0,2,Completed,0
11552,51,2002-08-26,Grand Slam,Outdoor,Hard,1st Round,5,Amritraj P.,Srichaphan P.,1024.0,33.0,0,3,Completed,0
12007,39,2007-07-10,International,Outdoor,Grass,1st Round,3,Bozoljac I.,Amritraj P.,142.0,467.0,1,2,Completed,0


In [609]:
X_Amritraj = data[(data['First_player']=='Amritraj P.')|(data['Second_player']=='Amritraj P.')]\
.drop(columns=['First_won', 'First_Sets', 'Second_Sets', 'ATP', 'Comment'])
y_Amritraj = data['First_won'][(data['First_player']=='Amritraj P.')|(data['Second_player']=='Amritraj P.')]

In [610]:
X_train, X_test, y_train, y_test = train_test_split(X_Amritraj, y_Amritraj, test_size=0.1, random_state=420)

In [611]:
model.get_feature_importance()

array([6.87303803e+01, 1.96170977e+01, 6.35609956e+00, 1.41115570e-01,
       0.00000000e+00, 3.83183990e-01, 0.00000000e+00, 0.00000000e+00,
       1.51619621e-02, 4.75696091e+00])

In [612]:
X_Amritraj

,Date,Series,Court,Surface,Round,Best of,First_player,Second_player,First_Rank,Second_Rank
781,2007-07-13,International,Outdoor,Grass,Quarterfinals,3,Amritraj P.,Mahut N.,467.0,83.0
1758,2007-07-12,International,Outdoor,Grass,2nd Round,3,Jenkins S.,Amritraj P.,367.0,467.0
2398,2006-01-04,International,Outdoor,Hard,2nd Round,3,Amritraj P.,Stepanek R.,267.0,20.0
3606,2008-07-12,International,Outdoor,Grass,Semifinals,3,Dancevic F.,Amritraj P.,99.0,305.0
4681,2003-07-07,International,Outdoor,Grass,1st Round,3,Amritraj P.,Ginepri R.,414.0,54.0
5289,2006-01-02,International,Outdoor,Hard,1st Round,3,Mello R.,Amritraj P.,111.0,267.0
6715,2003-07-22,International,Outdoor,Hard,1st Round,3,Roitman S.,Amritraj P.,140.0,396.0
10777,2003-07-30,International,Outdoor,Hard,2nd Round,3,Amritraj P.,Carlsen K.,351.0,76.0
11552,2002-08-26,Grand Slam,Outdoor,Hard,1st Round,5,Amritraj P.,Srichaphan P.,1024.0,33.0
12007,2007-07-10,International,Outdoor,Grass,1st Round,3,Bozoljac I.,Amritraj P.,142.0,467.0


In [613]:
y_Amritraj

781      0
1758     0
2398     0
3606     0
4681     0
5289     0
6715     0
10777    0
11552    0
12007    0
16578    0
19113    0
21303    0
24109    0
27239    1
28218    0
28753    0
30836    0
31694    1
34406    0
35042    1
35586    0
37971    0
40477    0
41998    0
42923    1
43094    0
43582    0
45755    0
45838    0
45911    0
46582    0
48074    0
50346    1
55459    0
Name: First_won, dtype: int64

In [614]:
model = catboost.CatBoostClassifier(depth=15,
                           learning_rate=1,
                           n_estimators=50,
                           loss_function='Logloss',
                           verbose=True)

In [615]:
model.fit(X_train, y_train, cat_features=[1,2,3,4,5,6,7])

0:	learn: 0.3977369	total: 4.55ms	remaining: 223ms
1:	learn: 0.3004151	total: 6.39ms	remaining: 153ms
2:	learn: 0.1998513	total: 6.86ms	remaining: 107ms
3:	learn: 0.1452511	total: 8.06ms	remaining: 92.8ms
4:	learn: 0.1070676	total: 9.28ms	remaining: 83.5ms
5:	learn: 0.0915561	total: 10.5ms	remaining: 76.9ms
6:	learn: 0.0841276	total: 10.9ms	remaining: 67ms
7:	learn: 0.0680557	total: 11.4ms	remaining: 59.7ms
8:	learn: 0.0604764	total: 12.1ms	remaining: 55ms
9:	learn: 0.0540349	total: 12.7ms	remaining: 50.9ms
10:	learn: 0.0512515	total: 14.9ms	remaining: 53ms
11:	learn: 0.0471126	total: 16.6ms	remaining: 52.4ms
12:	learn: 0.0434079	total: 17.2ms	remaining: 49ms
13:	learn: 0.0405317	total: 19ms	remaining: 49ms
14:	learn: 0.0381075	total: 20.8ms	remaining: 48.5ms
15:	learn: 0.0360276	total: 21.8ms	remaining: 46.4ms
16:	learn: 0.0346146	total: 23ms	remaining: 44.6ms
17:	learn: 0.0333269	total: 24.6ms	remaining: 43.8ms
18:	learn: 0.0324863	total: 26.9ms	remaining: 44ms
19:	learn: 0.0306861	t

In [616]:
cv_dataset = Pool(data=X_train,
                  label=y_train,
                  cat_features=[1,2,3,4,5,6,7])

params = {'depth':15,
          'learning_rate':1,
          'n_estimators':50,
          'loss_function':'Logloss',
          'verbose':True}

scores = cv(cv_dataset,
            params,
            fold_count=3)


0:	learn: 0.1703242	test: 0.2400273	best: 0.2400273 (0)	total: 20.4ms	remaining: 1000ms
1:	learn: 0.0564037	test: 0.1979713	best: 0.1979713 (1)	total: 41.2ms	remaining: 988ms
2:	learn: 0.0377997	test: 0.1880853	best: 0.1880853 (2)	total: 62ms	remaining: 971ms
3:	learn: 0.0266462	test: 0.2032414	best: 0.1880853 (2)	total: 83ms	remaining: 954ms
4:	learn: 0.0193961	test: 0.1992798	best: 0.1880853 (2)	total: 106ms	remaining: 955ms
5:	learn: 0.0144509	test: 0.1947538	best: 0.1880853 (2)	total: 128ms	remaining: 941ms
6:	learn: 0.0127590	test: 0.1936003	best: 0.1880853 (2)	total: 150ms	remaining: 919ms
7:	learn: 0.0117012	test: 0.1990142	best: 0.1880853 (2)	total: 172ms	remaining: 904ms
8:	learn: 0.0107811	test: 0.1985763	best: 0.1880853 (2)	total: 196ms	remaining: 892ms
9:	learn: 0.0089995	test: 0.1914134	best: 0.1880853 (2)	total: 220ms	remaining: 881ms
10:	learn: 0.0085313	test: 0.1849706	best: 0.1849706 (10)	total: 243ms	remaining: 863ms
11:	learn: 0.0080236	test: 0.1816816	best: 0.181681

In [617]:
scores

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.240027,0.174139,0.170324,0.061638
1,1,0.197971,0.268624,0.056404,0.029172
2,2,0.188085,0.274085,0.037800,0.020772
3,3,0.203241,0.312205,0.026646,0.010245
4,4,0.199280,0.308058,0.019396,0.006339
5,5,0.194754,0.303927,0.014451,0.004967
6,6,0.193600,0.301157,0.012759,0.004143
7,7,0.199014,0.311662,0.011701,0.003669
8,8,0.198576,0.311830,0.010781,0.003323
9,9,0.191413,0.303563,0.008999,0.001714


In [618]:
model.predict_proba(X_test)

array([[0.99356267, 0.00643733],
       [0.98769901, 0.01230099],
       [0.9816843 , 0.0183157 ],
       [0.99365635, 0.00634365]])

In [619]:
y_test

43582    0
10777    0
35586    0
48074    0
Name: First_won, dtype: int64

In [620]:
model.get_feature_importance()

array([6.87303803e+01, 1.96170977e+01, 6.35609956e+00, 1.41115570e-01,
       0.00000000e+00, 3.83183990e-01, 0.00000000e+00, 0.00000000e+00,
       1.51619621e-02, 4.75696091e+00])

In [621]:
data.dtypes

ATP                       int64
Date             datetime64[ns]
Series                   object
Court                    object
Surface                  object
Round                    object
Best of                   int64
First_player             object
Second_player            object
First_Rank              float64
Second_Rank             float64
First_Sets               object
Second_Sets              object
Comment                  object
First_won                 int64
dtype: object